# Assignment

In this assignment, we will use the dolphin network. The data is in "./data" folder. There are two csv files, i.e., node_table.csv and edge_table.csv. 
Load the network and complete the following assignments. 

In [19]:
import pandas as pd
import igraph
import numpy as np
from scipy import sparse

node_table = pd.read_csv("../data/node_table.csv")
edge_table = pd.read_csv("../data/edge_table.csv")
src, trg = tuple(edge_table[["src", "trg"]].values.T)
edge_list = tuple(zip(src, trg))

# node_id and name dictionary
n_nodes = node_table.shape[0]

# Construct the igraph object
g = igraph.Graph(edge_list, directed=False)

# Construct the adjacency matrix
A = g.get_adjacency_sparse()

# Node's group attribute
group_labels, group_membership = np.unique(
    node_table["sex"].values, return_inverse=True
)

**Question 1: Implement the following function to compute the degree assortativity of the network. Do not use the igraph.assortativity and g.assortativity_degree APIs.**

In [22]:
from scipy import stats


def degree_assortativity(A):
    """Compute the degree assortativity

    A: scipy.sparse adjacency matrix of a network

    Return:
    degree assortativity of the network.
    """
    src, trg, _ = sparse.find(sparse.triu(A, 1))
    deg = np.array(A.sum(axis=0)).ravel()
    return stats.pearsonr(deg[src], deg[trg])[0]


# Test
np.isclose(degree_assortativity(A), g.assortativity_degree(), 1e-2)

True

**Question 2: Implement the following function and compute the modularity for the "sex" node attribute.**

In [ ]:
def calc_modularity(A, membership):
    """
    Calculate the modularity of a graph given the membership of the nodes.
    A: CSR representation of the adjacency matrix
    membership: 1D array of integers

    Return the modularity value of the given membership
    """
    Q = ...
    return Q


# Test
assert np.isclose(calc_modularity(A, membership), g.modularity(membership), 1e-2)

**Question 3: Implement the Girvan-Newman algorithm**

Hint: In the previous assignment, we implemented a network attack based on betweenness centrality. You can recycle the most part of the code. Note that you need to remove an edge not a node.   

In [ ]:
def girvan_newman_algorithm(g):
    """
    Perform the Girvan-Newman algorithm on a graph.
    g: igraph Graph object

    Return the 1D array of membership of the nodes
    """
    membership = ...
    return membership


# Test
assert (
    g.community_edge_betweenness().as_clustering().membership
    == girvan_newman_algorithm(g)
)

**Question 4: Implement the following function and generate a network with 120 nodes and 20 communities by using the stochastic block model**

In [35]:
def generate_SBM_network(n_nodes, membership, P):
    """
    Generate a stochastic block model network
    n_nodes: number of nodes
    membership: 1D array of integers
    P: 2D array of connection probability between groups

    Return:
    net: CSR representation of the adjacency matrix generated by the SBM
    """
    # Generate the adjacency matrix
    Asbm = ...
    return Asbm


n_nodes = 120
K = 20
n_nodes_per_community = n_nodes // K
membership = np.arange(n_nodes) // n_nodes_per_community
P = np.ones(K) * (1.0 / n_nodes) + np.eye(K) * 0.9

net = generate_SBM_network(n_nodes=n_nodes, membership=membership, P=P)

In [39]:
# Test
def test_sbm_network(net, membership, P):
    rows, cols = np.arange(len(membership), dtype=int), membership
    membership_list = np.unique(membership)

    nrows, ncols = net.shape[0], len(membership_list)
    U = sparse.csr_matrix(
        (np.ones_like(rows), (rows, cols)),
        shape=(nrows, ncols),
    )
    E = U.T @ net @ U
    Nc = np.array(U.sum(axis=0)).reshape(-1)
    E = E.toarray()
    Pest = E / np.outer(Nc, Nc)
    Pest[np.isnan(Pest)] = 0.0
    assert np.sqrt(np.mean((Pest - P) ** 2)) <= 0.1


test_sbm_network(net, membership, P)